In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install bs4
!pip install html5lib
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=chrome_options)

In [ ]:
!pip install selenium

In [ ]:
!pip install selenium_stealth

In [ ]:
import pandas as pd

In [ ]:
import requests
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import html5lib
import re
import selenium
import os
# import the Keys class
from selenium.webdriver.common import keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.support.ui import Select
# import org.openqa.selenium.support.ui.Select
from collections import defaultdict
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36')
# options.add_argument("--no-sandbox")

In [ ]:
def datafromsinglestate(statename):
  url= "https://npiprofile.com/recent/"+statename
  wd = webdriver.Chrome(options=options)
  stealth(wd,
          languages=["en-US", "en"],
          vendor="Google Inc.",
          platform="Win32",
          webgl_vendor="Intel Inc.",
          renderer="Intel Iris OpenGL Engine",
          fix_hairline=True,
          )

  wd.get(url)
  time.sleep(10)
  select = Select(wd.find_element_by_name('table-recently-updated-'+statename+'_length'))
  select.select_by_value('-1')
  data = []
  soup = BeautifulSoup(wd.page_source, 'html.parser')
  table = soup.find("table", {"id": "table-recently-updated-"+statename})
  table_body = table.find('tbody')

  rows = table_body.find_all('tr')
  for row in rows:
      cols = row.find_all('td')
      cols = [ele.text.strip() for ele in cols]
      data.append([ele for ele in cols if ele])
  # print(data[4551])
  Npi = []
  Name = []
  Type = []
  Texonomy = []
  Address = []
  Updated_on = []
  count = 0

  for i in data:

    # print(count)
    count = count + 1
    if len(i) == 6:
      # print(count,i)
      Npi.append(i[0])
      Name.append(i[1])
      Type.append(i[2])
      Texonomy.append(i[3])
      Address.append(i[4])
      Updated_on.append(i[5])

  df = pd.DataFrame(data, columns = ['Npi','Name' ,'Type' ,'Texonomy', 'Address','Updated_on'])
  df['Updated_on'] = df['Updated_on'].astype('datetime64[ns]')
  # df1 = df[df['Texonomy'] == 'Home Health' ]
  from datetime import datetime

  df1 = df[df['Updated_on'] >= '2022-04-12' ]
          #  or df['Texonomy'] == 'Social Worker(Clinical)']
          # df['Texonomy'] == 'Home Health'
  print(df1.info())
  df1.to_csv(statename+'.csv', index=False)
  return df1

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dfnpi=pd.read_excel('/content/drive/MyDrive/Tahira NPI.xlsx')
npi = dfnpi['NPI'].tolist()
len(npi)

999

In [ ]:
name_adress = []
phone = []
url1= "https://npiregistry.cms.hhs.gov/provider-view/"
wd = webdriver.Chrome(options=options)
# get method to launch the URL
# wd.get("https://www.softwaretestingmaterial.com/sample-webpage-to-automate/")
count = 0

for i in npi[0:2]:
  try:
    wd = webdriver.Chrome(options=options)
    stealth(wd,
          languages=["en-US", "en"],
          vendor="Google Inc.",
          platform="Win32",
          webgl_vendor="Intel Inc.",
          renderer="Intel Iris OpenGL Engine",
          fix_hairline=True,
          )
    count = count + 1
    print(count)
    url = url1+ str(i)
# print(url)
    wd.get(url)
  #datafromsinglestate("fl-florida")
    time.sleep(1)
    soup = BeautifulSoup(wd.page_source)
    table = soup.body.find('table', attrs={'class': 'table table-striped table-bordered table-light'})
    section = table.find_all("td")
    print(section)
#ph = section.find("td", "phone")
    X = (section[13].text.split('|')[0].split("Phone:")[-1])
#print(type(X))
    print(X.strip())
    phone.append(X.strip())
  except:
    phone.append("none")



In [ ]:
print(len(phone))
dfnpi["Phone #"] = phone
dfnpi

In [ ]:
dfnpi.to_csv('/content/drive/MyDrive/Tahira_npi_with_numbers.csv', index=False)